# Configuring and Running Games

## Configuring Games

Setting up a custom poker game consists of providing rules for dealing, stakes and showdown evaluation through classes listed in {ref}`this <rules_api_reference>` section of the API Reference.

By default, when you set up a simple game like

In [3]:
from maverick import Game

game = Game(small_blind=10, big_blind=20, max_hands=2)

the library sets up a No-Limit Texas Hold'em game and is equivalent to the more detailed configuration

In [4]:
from maverick.rules import PokerRules, DealingRules, StakesRules, ShowdownRules

rules = PokerRules(
    name="NLHE",
    dealing=DealingRules(
        variant="texas_holdem",
        hole_cards=2,
    ),
    stakes=StakesRules(
        small_blind=10,
        big_blind=20,
        ante=0,
    ),
    showdown=ShowdownRules(
        hole_cards_required=0,
    ),
)

game = Game(rules=rules, max_hands=40)

In [5]:
game.rules == Game(small_blind=10, big_blind=20, max_hands=2).rules

True

This would be a configuration of a No-Limit Omaha game:

In [6]:
rules = PokerRules(
    name="Omaha",
    dealing=DealingRules(
        variant="omaha",
        hole_cards=4,
    ),
    stakes=StakesRules(
        small_blind=10,
        big_blind=20,
        ante=0,
    ),
    showdown=ShowdownRules(
        hole_cards_required=2,
    ),
)

game = Game(rules=rules, max_hands=40)

The {class}`~maverick.rules.PokerRules` class is also a subclass of PyDantic's {class}`~pydantic.main.BaseModel` class.

In [7]:
game.rules.model_dump()

{'name': 'Omaha',
 'dealing': {'variant': <GameVariant.OMAHA: 'omaha'>,
  'max_players': 9,
  'min_players': 2,
  'hole_cards': 4,
  'board_cards_total': 5,
  'board_deal_pattern': {<Street.PRE_FLOP: 0>: 0,
   <Street.FLOP: 1>: 3,
   <Street.TURN: 2>: 1,
   <Street.RIVER: 3>: 1}},
 'stakes': {'small_blind': 10, 'big_blind': 20, 'ante': 0},
 'showdown': {'hole_cards_required': 2},
 'rules_version': '1.0'}

### Limitations

1) You can only set up No-Limit games.
2) You can only set up Texas Hold'em or Omaha games. This limitation is not very restrictive, as it only influences the dealing semantics. This freezes the types of streets played and their order, but still leaves you with plenty of freedom. For instance, it doesn't stop you from defining a game where players have 3 private cards and in that case it would be considered as a special version of Omaha.
3) There is only one type of ante behaviour, which is that all players place the ante amount. This is not the only type of ante used in poker events such as WSOP, but this is the only one implemented in the library.

## Running Games

At this point you should be familiar with calling the `start` method of an instance of {class}`~maverick.game.Game` and you know that it drains the event queue. If you want more control, you can also play a game one event at a time using step-by-step execution. 

### Step-by-Step Execution of a Game

In [8]:
from maverick import (
    PlayerLike,
    PlayerState,
    GameEventType
)
from maverick.players import FoldBot, CallBot, AggressiveBot
import logging

# Configure logging such that we only get the log messages of the game
logging.basicConfig(level=logging.INFO, format="%(name)s: %(message)s", force=True)
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("maverick").setLevel(logging.INFO)

# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=40)

# Create and add players with different strategies
players: list[PlayerLike] = [
    CallBot(name="CallBot", state=PlayerState(stack=1000)),
    AggressiveBot(name="AggroBot", state=PlayerState(stack=1000)),
    FoldBot(name="FoldBot", state=PlayerState(stack=1000)),
]

for player in players:
    game.add_player(player)

game._initialize_game()
game._event_queue.append(GameEventType.GAME_STARTED)
print("Game initialized and GAME_STARTED event queued")
print(f"Has pending events: {game.has_events()}")

maverick: Player CallBot joined the game.
maverick: Player AggroBot joined the game.
maverick: Player FoldBot joined the game.


Game initialized and GAME_STARTED event queued
Has pending events: True


You can see the pending events:

In [9]:
game._event_queue

deque([<GameEventType.GAME_STARTED: 1>])

Consume one event:

In [10]:
_ = game.step()

maverick: ============================== Hand 1 ==============================



In [11]:
print(f"Has pending events: {game.has_events()}")

Has pending events: True


In [12]:
game._event_queue

deque([<GameEventType.HAND_STARTED: 2>])

Looking at the game state, you can tell that the deck is already initialized, but the players don't have their private cards (the holding) yet.

In [13]:
game.state.model_dump()

{'state_type': <GameStateType.STARTED: 3>,
 'street': <Street.PRE_FLOP: 0>,
 'players': [<maverick.players.callbot.CallBot at 0x113aff170>,
 'active_players': [],
 'current_player_index': 0,
 'deck': {'cards': [{'suit': <Suit.CLUBS: 'C'>, 'rank': <Rank.THREE: 3>},
   {'suit': <Suit.DIAMONDS: 'D'>, 'rank': <Rank.EIGHT: 8>},
   {'suit': <Suit.HEARTS: 'H'>, 'rank': <Rank.FIVE: 5>},
   {'suit': <Suit.SPADES: 'S'>, 'rank': <Rank.KING: 13>},
   {'suit': <Suit.DIAMONDS: 'D'>, 'rank': <Rank.SIX: 6>},
   {'suit': <Suit.SPADES: 'S'>, 'rank': <Rank.SIX: 6>},
   {'suit': <Suit.HEARTS: 'H'>, 'rank': <Rank.SIX: 6>},
   {'suit': <Suit.SPADES: 'S'>, 'rank': <Rank.FIVE: 5>},
   {'suit': <Suit.DIAMONDS: 'D'>, 'rank': <Rank.FIVE: 5>},
   {'suit': <Suit.DIAMONDS: 'D'>, 'rank': <Rank.TEN: 10>},
   {'suit': <Suit.SPADES: 'S'>, 'rank': <Rank.TEN: 10>},
   {'suit': <Suit.CLUBS: 'C'>, 'rank': <Rank.SEVEN: 7>},
   {'suit': <Suit.DIAMONDS: 'D'>, 'rank': <Rank.ACE: 14>},
   {'suit': <Suit.HEARTS: 'H'>, 'rank': <R